Cuando se me presentó el proyecto relacionado con los ataque de tiburones, se me vinieron a la cabeza, 3 primos mios, los cuales son unos apasionados, y ya se han hecho alguna escapada para surfear la olas, y por ello les pregunté ¿En que  pais os daría más pánico entrar al agua por la gran cantidad de tiburones?
Uno dijo EEUU, -Y otro contestó-, El pais mas peligroso sin duda es ISLAS REUNION, poca gente se atreve a sufear!

## HIPOTESIS

### ¿Son los ataques de tiburones de Isla Reunion, los mas mortales del mundo?

Importamos las formulas a utilizar y el CSV

In [3]:
#IMPORTAMOS PANDAS
import pandas as pd
import re 

In [4]:
#IMPORTAMOS EL CSV
df = pd.read_csv("/Users/santiagobermejo/Desktop/IRONHACK/Proyectos-Ironhack-0120/input/GSAF5.csv")


##### Comenzamos la investigación limpiando "df" con el fin de hacer el estudio lo mas preciso posible

In [5]:
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

###### Corregimos alguno errores encontrados en los nombres de las columnas:

In [6]:
sharkys = df.rename(columns={'Sex ':'Sex','Species ':'Species','Fatal (Y/N)':'Fatal'})

###### Nos fijamos en las columnas que son tendentes a aportar poca información y en caso de resultar poco utiles las eliminaremos

In [7]:
nullvals = sharkys.isnull().sum()[sharkys.isnull().sum() > 0]
nullvals

Country                     43
Area                       402
Location                   496
Activity                   527
Name                       200
Sex                        567
Age                       2681
Injury                      27
Fatal                       19
Time                      3213
Species                   2934
Investigator or Source      15
href formula                 1
href                         3
Unnamed: 22               5991
Unnamed: 23               5990
dtype: int64

###### Eliminamos las columnas "Unnamed: 22" y "Unnamed: 23 puesto que sus valores son NaN, del mismo modo eliminamos todos aquellos que consideramos que carecen de informacion, como 'href formula' y 'href' ya que en el PDF se muestra la misma informacion relevante.

In [8]:
sharkys = sharkys.drop(columns = ["Unnamed: 23", "Unnamed: 22","href formula","href"]) 

In [9]:
sharkys.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal', 'Time', 'Species',
       'Investigator or Source', 'pdf', 'Case Number.1', 'Case Number.2',
       'original order'],
      dtype='object')

###### Observamos que tenemos 3 columnas en las cuales se indican fechas

In [10]:
sharkys[["Case Number","Case Number.1","Case Number.2"]].head(10)

,Case Number,Case Number.1,Case Number.2
0,2016.09.18.c,2016.09.18.c,2016.09.18.c
1,2016.09.18.b,2016.09.18.b,2016.09.18.b
2,2016.09.18.a,2016.09.18.a,2016.09.18.a
3,2016.09.17,2016.09.17,2016.09.17
4,2016.09.15,2016.09.16,2016.09.15
5,2016.09.15.R,2016.09.15.R,2016.09.15.R
6,2016.09.11,2016.09.11,2016.09.11
7,2016.09.07,2016.09.07,2016.09.07
8,2016.09.06,2016.09.06,2016.09.06
9,2016.09.05.b,2016.09.05.b,2016.09.05.b


###### Como se puede comprobar, son columnas iguales, nos quedamos en este caso con "Case Number"

###### Por lo que nos quedamos con una de ellas y basamos la informacion en "Case Number" y "Year"

In [11]:
sharkys = sharkys.drop(columns = ["Date","Case Number.1","Case Number.2"])

###### Eliminamos aquellas filas que no nos apaortan informacion de fecha ya que difucltaría el estudio

In [12]:
tibus = sharkys[~sharkys["Case Number"].str.contains('ND','nd')][:-5]
tibus.head()

,Case Number,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal,Time,Species,Investigator or Source,pdf,original order
0,2016.09.18.c,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16,Minor injury to thigh,N,13h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,5993
1,2016.09.18.b,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,36,Lacerations to hands,N,11h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,5992
2,2016.09.18.a,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43,Lacerations to lower leg,N,10h43,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,5991
3,2016.09.17,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,NaN,Struck by fin on chest & leg,N,NaN,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,5990
4,2016.09.15,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,5989


###### Ajustamos las fechas a un patron único, como en un a columna ya tenemos el "year", nos quedaremos con el "month" de la nueva columna

In [13]:
a = list(tibus["Case Number"])
b = ' '.join(a)
d = re.findall(r'\.\d{2}[\.-]', b)
d = [a.strip(".") for a in d]

(Nos daba una longitud inferior a la de nuestro DataFrame asi que comprobamos con un for para mostrar el valso que esta fllando y corregimos el regex de arriba)

In [14]:
años=[]
for y in a:
    if re.findall(r'\.\d{2}\.', y):
        años.append(re.findall(r'\.\d{2}\.', y))
    else:
        print(y)
len(años)

2015.07-10


5867

###### Comprobamos:

In [15]:
len(d)

5868

###### Cambiamos el nombre de los meses y de la columna:

In [16]:
Months = pd.DataFrame(d, columns = ["Months"])

In [17]:
Months.replace({
    '00':'UNKNOWN',
    '01':'January',
    '02':'February',
    '03':'March',
    '04':'April',
    '05':'May',
    '06':'June',
    '07':'July',
    '08':'August',
    '09':'September',
    '10':'October',
    '11':'November',
    '12':'December'
}, inplace = True)

len(Months)

5868

In [18]:
data_date = Months
tibus["Case Number"] = data_date

In [19]:
My_Data = tibus.rename(columns={"Case Number":"Months"})

###### Imprimimos nuestro nuevo DataFrame

In [20]:
My_Data.head()

,Months,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal,Time,Species,Investigator or Source,pdf,original order
0,September,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16,Minor injury to thigh,N,13h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,5993
1,September,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,36,Lacerations to hands,N,11h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,5992
2,September,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43,Lacerations to lower leg,N,10h43,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,5991
3,September,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,NaN,Struck by fin on chest & leg,N,NaN,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,5990
4,September,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,5989


###### Entramos en detalle de nuestra investigación seleccionando las columanas mas interesantes

In [21]:
dangerest = tibus[["Activity","Country","Species","Sex","Fatal"]]

###### Nos fijamos de nuevo en los valores nulos para ajustar la precision de la hipotesis

In [22]:
newnullvals = dangerest.isnull().sum()[dangerest.isnull().sum() > 0]
newnullvals

Activity     508
Country       39
Species     2844
Sex          557
Fatal         19
dtype: int64

###### Rellenamos NaN por valores válidos

In [23]:
NonNaN = {'Activity':'UNKNOWN',
          'Country':'UNKNOWN',
          'Area':'UNKNOWN',
          'Species':'UNKNOWN',
          'Sex':'UNKNOWN',
          'Age':'UNKNOWN',
          'Fatal':'UNKNOWN',
          'Time':'UNKNOWN'
          }
dangerest.fillna(value=NonNaN, inplace = True)
dangerest.head() #Para verificar se se han sustituido imprimimos una pequeña lista

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


,Activity,Country,Species,Sex,Fatal
0,Surfing,USA,UNKNOWN,M,N
1,Surfing,USA,UNKNOWN,M,N
2,Surfing,USA,UNKNOWN,M,N
3,Surfing,AUSTRALIA,UNKNOWN,M,N
4,Surfing,AUSTRALIA,2 m shark,M,N


In [24]:
dangerest.isnull().sum()

Activity    0
Country     0
Species     0
Sex         0
Fatal       0
dtype: int64

In [25]:
dangerest.columns

Index(['Activity', 'Country', 'Species', 'Sex', 'Fatal'], dtype='object')

###### Hago un bucle for para eliminar aquellos datos que contienen espacios

In [26]:
for col in dangerest:
    dangerest[col]=dangerest[col].str.strip()

<ipython-input-26-f26f3ed15955>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dangerest[col]=dangerest[col].str.strip()


###### Arreglamos aquellos datos relevantes que vayamos a usar

In [27]:
dangerest["Sex"].value_counts()

M          4736
F           572
UNKNOWN     557
.             1
lli           1
N             1
Name: Sex, dtype: int64

In [28]:
dangerest["Sex"].replace({"lli": "UNKNOWN", "N": "UNKNOWN",".": "UNKNOWN"},inplace=True)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/generic.py:6786: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [29]:
dangerest["Sex"].value_counts()

M          4736
F           572
UNKNOWN     560
Name: Sex, dtype: int64

In [30]:
dangerest["Fatal"].value_counts()

N          4258
Y          1495
UNKNOWN     112
F             1
#VALUE!       1
n             1
Name: Fatal, dtype: int64

In [31]:
dangerest["Fatal"].replace({"F": "UNKNOWN", "#VALUE!": "UNKNOWN","n": "N"},inplace=True)

In [32]:
dangerest["Fatal"].value_counts()

N          4259
Y          1495
UNKNOWN     114
Name: Fatal, dtype: int64

In [33]:
dangerest[["Country","Sex","Fatal"]].head()

,Country,Sex,Fatal
0,USA,M,N
1,USA,M,N
2,USA,M,N
3,AUSTRALIA,M,N
4,AUSTRALIA,M,N


##### Entramos al detalle de los paises con mas ataques 

In [34]:
y = dangerest[dangerest["Fatal"]=="Y"].groupby("Country").agg({"Fatal":"count"}).sort_values('Fatal', ascending = False)
y["TotalAttacks"] = dangerest[dangerest["Fatal"].isin(["Y","N"])].groupby("Country").agg({"Fatal":"count"}).sort_values('Fatal', ascending = False)
y["Percentage"] = y["Fatal"]/y["TotalAttacks"]
display(y.head(15))

,Fatal,TotalAttacks,Percentage
Country,,,
AUSTRALIA,333,1236,0.269417
USA,243,2071,0.117335
SOUTH AFRICA,130,551,0.235935
PAPUA NEW GUINEA,54,127,0.425197
MEXICO,43,82,0.524390
BRAZIL,40,100,0.400000
PHILIPPINES,38,58,0.655172
NEW ZEALAND,27,121,0.223140
REUNION,26,55,0.472727


### Observando los porcentajes se podría sacar la conclusión que el pais con los tiburones más mortales es PHILIPPINES, pero mis primos me llevan la contraria y me siguen diciendo que ahí se hace menos surf, que el más peligroso es ISLAS REUNION

###### Hago del mismo modo, un estudio según el sexo, para recomdar a mis primos si el viaje lo pueden hacer con sus mujeres.

In [35]:
y = dangerest[dangerest["Sex"]=="M"].groupby("Country").agg({"Sex":"count"}).sort_values('Sex', ascending = False)
y["TotalAttacks"] = dangerest[dangerest["Sex"].isin(["M","F"])].groupby("Country").agg({"Sex":"count"}).sort_values('Sex', ascending = False)
y["Percentage"] = y["Sex"]/y["TotalAttacks"]
display(y.head(15))

,Sex,TotalAttacks,Percentage
Country,,,
USA,1693,1998,0.847347
AUSTRALIA,1040,1119,0.929401
SOUTH AFRICA,469,493,0.951318
PAPUA NEW GUINEA,104,110,0.945455
NEW ZEALAND,97,109,0.889908
BRAZIL,85,88,0.965909
BAHAMAS,82,93,0.881720
MEXICO,63,74,0.851351
ITALY,55,57,0.964912


Como se puede observar, el porcentaje de mujeres que han sufrido ataques es muy bajo, ¡¡Pueden acompañarlas!!

###### Procedo a hacer el estudio del pais detallando la actividad "Surfing"

In [36]:
surfingAttack = dangerest[dangerest.Activity == "Surfing"]

###### Hago el mismo estudio que antes pero solo con la actividad indicada

In [37]:
z = surfingAttack[surfingAttack["Fatal"]=="Y"].groupby("Country").agg({"Fatal":"count"}).sort_values('Fatal', ascending = False)
z["TotalAttacks"] = surfingAttack[surfingAttack["Fatal"].isin(["Y","N"])].groupby("Country").agg({"Fatal":"count"}).sort_values('Fatal', ascending = False)
z["Percentage"] = z["Fatal"]/z["TotalAttacks"]
display(z.head(10))

,Fatal,TotalAttacks,Percentage
Country,,,
AUSTRALIA,13,168,0.077381
USA,11,530,0.020755
SOUTH AFRICA,8,93,0.086022
REUNION,7,20,0.350000
BRAZIL,4,35,0.114286
MEXICO,3,7,0.428571
COSTA RICA,1,3,0.333333
EL SALVADOR,1,2,0.500000
NEW CALEDONIA,1,2,0.500000


### Ahora, la cosa cambia, Mexico obitene el porcentaje mas alto, pero con un numero de ataque muy bajo, asi que mis primos estaban en lo cierto, ISLAS REUNION el es pais con los tiburones mas mortales del mundo

###### Y continuo el estudio centrandome solo en ISLA REUNION

In [38]:
sharkys.columns

Index(['Case Number', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal', 'Time', 'Species',
       'Investigator or Source', 'pdf', 'original order'],
      dtype='object')

In [39]:
#Detalle de REUNION
estudio_reunion = tibus[tibus.Country == "REUNION"]
estudio_IS = estudio_reunion[['Case Number','Type', 'Area', 'Location','Activity', 'Sex', 'Age', 'Injury', 'Fatal', 'Time', 'Species']]

In [40]:
NaNout = {'Activity':'UNKNOWN',
          'Area':'UNKNOWN',
          'Species':'UNKNOWN',
          'Sex':'UNKNOWN',
          'Location':'UNKNOWN',
          'Age':'UNKNOWN',
          'Fatal':'UNKNOWN',
          'Injury':'UNKNOWN',
          'Time':'UNKNOWN'
          }
estudio_IS.fillna(value=NaNout, inplace = True)


In [41]:
estudio_IS.isnull().sum()

Case Number    0
Type           0
Area           0
Location       0
Activity       0
Sex            0
Age            0
Injury         0
Fatal          0
Time           0
Species        0
dtype: int64

In [42]:
c = estudio_IS[estudio_IS["Fatal"]=="Y"].groupby("Activity").agg({"Fatal":"count"}).sort_values('Fatal', ascending = False)
c["TotalAttacks"] = estudio_IS[estudio_IS["Fatal"].isin(["Y","N"])].groupby("Activity").agg({"Fatal":"count"}).sort_values('Fatal', ascending = False)
c["Percentage"] = c["Fatal"]/c["TotalAttacks"]
display(c)

,Fatal,TotalAttacks,Percentage
Activity,,,
Surfing,7,20,0.350000
Spearfishing,3,7,0.428571
Body boarding,2,8,0.250000
Swimming,2,2,1.000000
UNKNOWN,2,3,0.666667
Bathing,1,1,1.000000
Body-boarding,1,1,1.000000
Boogie Boarding,1,1,1.000000
Canoe with 3 men onboard sank,1,1,1.000000


###### Con este estudio les recominedo que no vayan a ISLAS REUNION a hacer surf, y que vayan a USA, para ello me piden recomendaciones de fechas y lugares

In [43]:
estudio_estadosUnidos = My_Data[My_Data.Country == "USA"]
surfingAttackUSA = My_Data[My_Data.Activity == "Surfing"]
estudio_USA_surfing = surfingAttackUSA[['Months', 'Area', 'Location','Activity', 'Sex', 'Fatal', 'Species']]
estudio_USA_surfing

,Months,Area,Location,Activity,Sex,Fatal,Species
0,September,Florida,"New Smyrna Beach, Volusia County",Surfing,M,N,NaN
1,September,Florida,"New Smyrna Beach, Volusia County",Surfing,M,N,NaN
2,September,Florida,"New Smyrna Beach, Volusia County",Surfing,M,N,NaN
3,September,Victoria,Thirteenth Beach,Surfing,M,N,NaN
4,September,Victoria,Bells Beach,Surfing,M,N,2 m shark
...,...,...,...,...,...,...,...
4687,March,New South Wales,Newcastle,Surfing,M,N,NaN
5004,January,New South Wales,"Stockton Beach, Newcastle",Surfing,M,Y,NaN
5048,March,New South Wales,Newcastle,Surfing,M,N,12' shark
5808,UNKNOWN,Hawaii,"Uo, Lahaina, Maui",Surfing,M,Y,NaN


In [44]:
NaNout = {'Activity':'UNKNOWN',
          'Area':'UNKNOWN',
          'Species':'UNKNOWN',
          'Sex':'UNKNOWN',
          'Location':'UNKNOWN',
          'Age':'UNKNOWN',
          'Fatal':'UNKNOWN',
          'Injury':'UNKNOWN',
          'Time':'UNKNOWN'
          }
estudio_USA_surfing.fillna(value=NaNout, inplace = True)

In [45]:
estudio_USA_surfing.isnull().sum()

Months      0
Area        0
Location    0
Activity    0
Sex         0
Fatal       0
Species     0
dtype: int64

###### Hago el estudio por areas para prohibirles los mas peligrosos

In [46]:
w = estudio_USA_surfing[estudio_USA_surfing["Fatal"]=="Y"].groupby("Area").agg({"Fatal":"count"}).sort_values('Fatal', ascending = False)
w["TotalAttacks"] = estudio_USA_surfing[estudio_USA_surfing["Fatal"].isin(["Y","N"])].groupby("Area").agg({"Fatal":"count"}).sort_values('Fatal', ascending = False)
w["Percentage"] = w["Fatal"]/w["TotalAttacks"]
display(w)

,Fatal,TotalAttacks,Percentage
Area,,,
Eastern Cape Province,6,55,0.109091
Hawaii,6,52,0.115385
Western Australia,5,19,0.263158
California,4,71,0.056338
South Australia,4,22,0.181818
Pernambuco,4,30,0.133333
New South Wales,3,88,0.034091
Western Cape Province,2,24,0.083333
Guerro,2,3,0.666667


###### Y con esto les prohibo ir a Eastern Cape Province, Hawaii, Western Australia y South Australia

###### Finalmente me piden el mes del año mas recomndable para evitar ataques de tiburones haciendo surf en estados unidos:

In [47]:
w = estudio_USA_surfing[estudio_USA_surfing["Fatal"]=="Y"].groupby("Months").agg({"Fatal":"count"}).sort_values('Fatal', ascending = False)
w["TotalAttacks"] = estudio_USA_surfing[estudio_USA_surfing["Fatal"].isin(["Y","N"])].groupby("Months").agg({"Fatal":"count"}).sort_values('Fatal', ascending = False)
w["Percentage"] = w["Fatal"]/w["TotalAttacks"]
display(w)

,Fatal,TotalAttacks,Percentage
Months,,,
July,7,94,0.074468
April,6,72,0.083333
December,5,52,0.096154
January,5,51,0.098039
September,5,109,0.045872
August,4,85,0.047059
March,4,70,0.057143
May,4,57,0.070175
June,3,63,0.047619


## SIN DUDA OCTUBRE ES EL MEJOR MES PARA IR!!!